In [ ]:
import numpy as np
import regex as re
import time

In [ ]:
!pip install pyaspeller --quiet
import pyaspeller

     |████████████████████████████████| 63 kB 1.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
def splitStrNum(text):
    '''
        text - слово для проверки
        разделяет текст типа "live48" на "live 48"
    '''
    tmp1 = re.findall("[0-9]+",text)
    tmp2 = re.findall("[^0-9]+",text)
    tmp2 = "".join(tmp2)
    tmp1 = "".join(tmp1)
    order_flag = 0
    for i in text:
        if i.isalpha():
            order_flag = 0
            break
        elif i.isdigit():
            order_flag = 1
            break
    # print(tmp1,tmp2)
    if tmp1 == "":
        text = tmp2
    else:
        if order_flag == 0:
            text = tmp2 + " " + tmp1
        else:
            text = tmp1 + " " + tmp2
    return text

In [ ]:
def notRusInWord(word) -> bool:
    '''
        word - входной текст
        возвращает True, если в слове нет русских букв
        иначе False 
    '''
    return (not True in [a.lower() in 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя' for a in word])

In [ ]:
def checkZeroWeightRight(speller,word,spelled_word,debug=False) -> bool:
    '''
        speller - корректор ошибок
        word - строка для проверки
        spelled_word - исправленное оригинальное слово
        debug - режим отладки
        проверяет слово, если у него скор меньше порогового 
        (например у "йцукенг" скор 0.25) 
    '''
    start = time.time()
    not_rus_word = notRusInWord(word.lower())
    not_rus_spelled_word = notRusInWord(spelled_word.lower())
    if debug:
        print(not_rus_word==not_rus_spelled_word)
    word_on_eng = transliter_rus2eng(spelled_word,not_rus_word!=not_rus_spelled_word,debug)    
    try:
        word_back = speller.spelled(word_on_eng)
    except:
        word_back = word_on_eng
    word_back = "".join(word_back.split(" "))
    
    spelled_word = "".join(spelled_word.split(" "))
    if word == word_on_eng:
        word_back = spelled_word    
    not_rus_back_word = notRusInWord(word_back.lower())
    not_rus_eng_word = notRusInWord(word_on_eng.lower()) 
    if debug:
        print(word_back,word,word_on_eng,spelled_word)
        print("checkZeroWeightRight: ",time.time() - start)
    return (word_back == spelled_word or (not_rus_eng_word != not_rus_back_word and word_on_eng == spelled_word))

In [ ]:
def checkCorrect(speller,text,return_weight,debug = False) -> float:
    '''     
        speller - корректор ошибок
        text - строка для проверки
        return_weight - если True, то вернет вес от 0 до 1, иначе вернет флаг фрода (0 или 1)
        debug - режим отладки (True - включить)
        возвращает 1, если ответ фрод
        иначе 0 
    '''
    start = time.time()
    if debug:
        print(text)
    default_threeshold = 0.551
    tmp_prepare = prepare_text(text)
    tmp = tmp_prepare
    # если сайт, то не фрод
    if ".ru" in text or ".com" in text or ".ру" in text:
        return 0
    tmp = tmp.replace(".","")
    # если длина алфавита - один симфол, то фрод
    if len(set(tmp.replace(" ",""))) <= 1:
        return 1
    # если только из чисел ответ, то фрод
    if True not in [w.isalpha() for w in tmp]:
        return 1
    judge = []
    if debug:
        print(tmp)
    for word in list(tmp.split(" ")):
        len_word = len(word)
        list_set_word = list(set(word))
        is_alpha_in_list = True in [i.isalpha() for i in list_set_word]
        if len(list_set_word) == 0:
            continue
        lstWords = []
        if True in [s.isdigit() for s in word]:
            word = splitStrNum(word)
        if debug:
            print("word:",word)
        
        oneLenWords = ['а','в','и','к','о','с','у','я','р']
        twoLenWords = ['из','вы','бы','гб','да','до','fm','фм'
                       'еж','за','их','кг','ко','мы',
                       'на','не','но','ну','об','он',
                       'от','по','ту','те','ты','та','то',
                       'уж','ух','уф','ус','че','шо','тв','tv','тц',
                       'ща','юг','яд','хз','ру','fm','ne','da','vk','вк','рф']
        # проверка на скудность алфавита (типа лалалала ркуркурку и тд)
        if (len_word/len(list_set_word) >= 2) and len_word > 6:
            judge.append(0.7)
        elif (len(list_set_word) == 2) and len_word > 4:
            judge.append(0.7)            
        elif len_word in range(3,4):
            judge.append(shortAnsCheck(speller,word,debug))
        elif len_word > 2 and is_alpha_in_list:
            long_ans_w, long_ans_l = checkLongWord(speller,word,len_word,debug)
            judge.append(long_ans_w)
            lstWords = long_ans_l
        elif word in oneLenWords:
            # добавлям дефолтный вес, если слово длинной 1
            judge.append(default_threeshold)
        elif word in twoLenWords:
            # добавлям дефолтный вес, если слово длинной 2
            judge.append(default_threeshold)   
        # иначе если слово не из букв алфавита, то добавляем дефолтный вес         
        elif not is_alpha_in_list:
            judge.append(default_threeshold+0.1)
        # иначе ставим вес чуть больше порогового
        else:
            judge.append(3/5)
        # дополнительная проверка слова
        if (judge[-1] <= 0.55 and len_word > 5) or (judge[-1] == 0 and len_word!=3):
            if debug:
                print('zero gravity:',lstWords)
            isZero = checkZeroWeightRight(speller,word,lstWords[-1],debug)
            if isZero:
                judge[-1] = 0.05
            else:
                judge[-1] = 0.95
    if len(judge) == 0:
        # дефолтный случай (вроде ниразу не было)
        retVal = speller.spelled(tmp)
        print("what")
        return retVal
    if debug:
        print(judge)
    # усредняем веса и это будет результирующий скор
    retVal = np.mean(judge)
    if debug:
        print("checkCorrect: ",time.time() - start)
    if return_weight:
        return retVal
    else:
        return 0 if retVal <= 0.55 else 1

In [ ]:
def checkLongWord(speller,word,len_word,debug):
    '''
        speller - корректор опечаток
        word - проверяемое слово
        len_word - длина слова
        debug - режим отладки
        проверяет слова, длиной больше трех 
    '''
    start = time.time()
    lstWords = []
    if debug:
        print(word)
    spelled_word = speller.spelled(word)
    if abs(len_word - len(spelled_word)) <= 2:
        word = spelled_word 
    # if len(word) <= 7:
    for i,symbol in enumerate(word):
        if symbol.isalpha() and len(lstWords) <= 5:
            # для случая, когда есть удвоенные буквы (они дадут одни и теже варианты, что не очень хорошо, как замечено)
            if i!=0 and i != len_word-1 and word[i] == word[i-1]:
                continue
            else:
                lstWords.append(word[:i]+word[i]+word[i:])

    # if len(word) >= 8:
    #     for i,symbol in enumerate(word):
    #         if i!=0 and i != len_word-1 and word[i] == word[i-1]:
    #             continue          
    #         elif len(lstWords) <= 5:
    #             lstWords.append(word[:i]+word[i+1:])
    #         else:
    #             break
    lstWords.append(word)  
    lst_index = len(lstWords) 
    count_equal_in_row = 0
    count_different_in_row = 0
    for i,elem in enumerate(lstWords):
        pos_var = speller.spelled(elem)
        if abs(len_word - len(pos_var)) < 3 or (pos_var in word) or (word in pos_var):
            if debug:
                print("checkLongWord variant ",elem,":",time.time() - start)
            lstWords[i] = speller.spelled(elem)
            if i!=0 and lstWords[i]!=lstWords[i-1]:
                count_different_in_row+=1
            else:
                count_different_in_row = 0
        if i!=0 and lstWords[i] == lstWords[i-1]:
            count_equal_in_row+=1
        if count_equal_in_row == 3 or count_different_in_row == 3:
            # если 4 подряд одинаковых варианта или 4 подряд разных, то остановка
            lst_index = i
            break
    if debug:
        print(lstWords[:lst_index])
    lstWords = lstWords[:lst_index]

    tmp_weight = (len(set(lstWords[:lst_index]))-1) / (len(lstWords[:lst_index])-1)
    
    if debug:
        print(tmp_weight)    
        print("checkLongWord: ",time.time() - start)
    return tmp_weight, lstWords

In [ ]:
def shortAnsCheck(speller,word,debug) -> float:
    '''
        speller - корректор опечаток
        word - проверяемое слово
        debug - режим отладки
        проверяет слово длиной 3
        волшебной палочкой добавляем слово "это" и "есть"
        для того, чтобы было больше "контекста"
        так проверка орфографии лучше рабоатет
        можно подтягивать текст вопроса, но лучше не надо
    '''
    start = time.time()
    temp_sym = ''
    for j in word:
        if j.isalpha():
            temp_sym = j
            break
    if temp_sym == '':
        return 0.6
    w1 = "это "
    w2 = "есть "
    temp_w1_3 = w1 + temp_sym + word
    temp_w1_3 = speller.spelled(temp_w1_3)
    temp_w1_3 = temp_w1_3.replace(w1,"")
    temp_w2_3 = w2 + temp_sym + word
    temp_w2_3 = speller.spelled(temp_w2_3)
    temp_w2_3 = temp_w2_3.replace(w2,"")    
    if debug:
        print(temp_w1_3,word,temp_sym + word)
        print(temp_w2_3,word,temp_sym + word)
    if (temp_w1_3 == word or (word in temp_w1_3 and temp_w1_3 != temp_sym + word))\
        or (temp_w2_3 == word or (word in temp_w2_3 and temp_w2_3 != temp_sym + word)):
        retVal = 0.25
    else:
        retVal = 0.75
    if debug:
        print("shortAnsCheck: ",time.time() - start)
    return retVal

In [ ]:
def transliter_rus2eng(text,diflang,debug):
    '''
        делает из привет -> ghbdtn
        или из ghbdtn -> привет (если текст на английском) 
    '''
    start = time.time()
    layout_dict = {
        'q':'й','w':'ц','e':'у','r':'к',
        't':'е','y':'н','u':'г','i':'ш',
        'o':'щ','p':'з','[':'х',']':'ъ',
        'a':'ф','s':'ы','d':'в','f':'а',
        'g':'п','h':'р','j':'о','k':'л',
        'l':'д',';':'ж',"'":'э','z':'я',
        'x':'ч','c':'с','v':'м','b':'и',
        'n':'т','m':'ь',',':'б','.':'ю'
    }    
    new_d = dict((v, k) for k, v in layout_dict.items())
    retVal = ""
    for i,t in enumerate(text):
        try:
            retVal+=(new_d[t])
        except:
            if diflang:
                try:
                    retVal+=layout_dict[t]
                except:
                    retVal+=(t)
            else:
                retVal+=(t)
    if debug:
        print("transliter_rus2eng: ",time.time() - start)
    return retVal

In [ ]:
def prepare_text(text):
    '''
        удаляет все символы кроме '.',',','!','?', 
        цифр и символов рус и англ алфавитов
        заменяет ё на е, 
    '''    
    # text = text.replace(".ru","")
    # text = text.replace(".ру","")
    text = text.replace("ё","е")
    # text = word_collapse(text,2)
    rule = "[1-9a-zA-Zа-яА-Я\s.!?,]+"
    res = " ".join(re.findall(rule, text))
    res = res.replace("."," ")
    res = res.replace(","," ")
    res = res.replace("!"," ")
    res = res.replace("?"," ")
    if res == "":
        res = "11111"
    elif res[-1] == " ":
        res = res[0:-1]

    while "  " in res:
        res = str.replace(res,"  "," ")
    return res

In [ ]:
speller = pyaspeller.YandexSpeller(lang='ru')

In [ ]:
checkCorrect(speller,"подтпе",True,True)

подтпе
подтпе
word: подтпе
подтпе
checkLongWord variant  пподтипе : 1.3863108158111572
checkLongWord variant  поодтипе : 2.7173702716827393
checkLongWord variant  поддтипе : 4.048262596130371
checkLongWord variant  подттипе : 5.299460172653198
['подтипе', 'подтипе', 'подтипе']
0.0
checkLongWord:  5.878225326538086
zero gravity: ['подтипе', 'подтипе', 'подтипе']
True
transliter_rus2eng:  2.0265579223632812e-05
подтипе подтпе gjlnbgt подтипе
checkZeroWeightRight:  0.6505413055419922
[0.05]
checkCorrect:  6.530706882476807


0.05

In [ ]:
speller.spelled("1исбет")

'1исбет'

In [ ]:
checkZeroWeightRight(speller,"1исбет","1исбет",True)

True
transliter_rus2eng:  3.2901763916015625e-05
1bc,tn 1исбет 1bc,tn 1исбет
checkZeroWeightRight:  0.6277258396148682


False

In [ ]:
checkCorrect(speller,"1иксбет",False,True)

1иксбет
1иксбет
word: 1 иксбет
1 иксбет
checkLongWord variant   иксбет : 0.5880241394042969
checkLongWord variant  1иксбет : 1.7997615337371826
checkLongWord variant  1 ксбет : 3.073932409286499
checkLongWord variant  1 исбет : 4.377570152282715
[' иксбет', '1иксбет', '1 ксбет']
1.0
checkLongWord:  5.017582178115845
[1.0]
checkCorrect:  5.021924257278442


1

In [ ]:
gen = speller.spell("троквезасин")
fst = next(gen)
fst['s']

['троксевазин', 'троксевасин', 'троквевазин']

In [ ]:
fst = next(gen)
fst['s']

['мгшв', 'вгшв', 'вгсв', 'вгв']

In [ ]:
speller = Hunspell()

In [ ]:
from textblob import TextBlob

In [ ]:
TextBlob

In [ ]:
a = TextBlob("не поял")

In [ ]:
a.correct()

TextBlob("of поял")